# Install relevant libaries and make the ETC configuration files

In [35]:
"""Install all relevant files, run only once!"""

!pip install matplotlib
!pip install scipy
!cd ../src && make -f Makefile all
!cd ../src && make -f Makefile.omp all
!cd .. && pip install .


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
cc -c -o gsetc.o -O3 -DHGCDTE_SUTR -DMOONLIGHT_ gsetc.c
cc gsetc.o -lm -o gsetc.x
cp gsetc.x ../python/pfsspecsim/bin/.
cc -c -o gsetc_omp.o -O3 -fopenmp -DHGCDTE_SUTR -DMOONLIGHT_ -DQUIET_FOR_OMP gsetc_omp.c
cc gsetc_omp.o -fopenmp -lm -o gsetc_omp.x
cp gsetc_omp.x ../python/pfsspecsim/bin/.
Defaulting to user installation because normal site-packages is not writeable
Processing /home/users/qai11/Documents/spt_ExposureTimeCalculator
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'pfsspecsim' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to t

# Import all libraries for running the ETC

In [32]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

"""Import the relevant libraries"""
import os
import sys
import matplotlib.pyplot as plt
import scipy
import numpy as np
import time
from pathlib import Path


# Add the path to the pfsspecsim module

# Go up one level to the project root
project_root = Path().resolve().parent

# Path to the pfsspecsim module
module_path = project_root / 'python' / 'pfsspecsim'

# Add it to sys.path if it exists
if module_path.exists():
    sys.path.insert(0, str(module_path))
else:
    raise FileNotFoundError(f"Module path not found: {module_path}")

# Import the pfsspecsim module
import pfsetc



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# How to run the exposure time calculator (ETC)

- Enter the Magnitude of the star in "magnitude" variable
- Enter the seeing conditions if known in "seeing" variable (Default = 1.5)
- If required change "target_wavelength" and "target_snr" (Default = 560nm and 550 S/R)

Code will take 2-5 minutes to run. If needed a S/R curve can be calculated in the next cell.


In [33]:
#Second test full run through
start_time = time.time()

#Define the functions for getting the S/N at a specific wavelength and finding the exposure time for a target S/N
def get_snr_at_wavelength(filename, target_wavelength, tolerance=0.5):
    data = np.loadtxt(filename)
    # Column 2 is wavelength, column 3 is S/N per pixel necessary for finding the exp time
    wavelengths = data[:, 2]
    snr_values = data[:, 3]
    
    # Find index of closest wavelength just in case the exact wavelength is not present
    idx = np.argmin(np.abs(wavelengths - target_wavelength))
    if abs(wavelengths[idx] - target_wavelength) <= tolerance:
        return snr_values[idx]
    else:
        return None

def find_min_exp_time_for_target_snr(mag=5.8, seeing=1.5, target_wavelength=560, 
                                     target_snr=550, step=100, min_exp=10, max_exp=1800):
    for exp_time in range(min_exp, max_exp + 1, step):
        # print(step)
        etc = pfsetc.Etc()
        etc.set_param('MAG_FILE', mag)
        etc.set_param('SEEING', seeing)
        etc.set_param('EXP_NUM', 1)
        etc.set_param('EXP_TIME', exp_time)
        etc.run()
        
        snr = get_snr_at_wavelength('out/ref.snc.dat', target_wavelength)
        
        if snr is None:
            print(f"Could not find S/N near {target_wavelength} nm.")
            continue
        
        print(f"EXP_TIME = {exp_time}s --> S/N = {snr:.2f}")
        
        if snr >= target_snr:
            print(f" Reached target S/N ≥ {target_snr} at {exp_time}s")
            return exp_time, snr
    
    print(f"Target S/N of {target_snr} not reached within {max_exp}s limit.")
    return max_exp, snr  # Fall back to max allowed time


# Change the Magnitude of the star in next cell

In [34]:
# Define the target wavelength and desired S/N
target_wavelength = 560  # in nm
# target_snr = 550 # Desired signal-to-noise ratio
target_snr = 850 #placeholder do not trust for real calculations, this is just to test the code and will be corrected later
seeing = 1.5  # in arcseconds based on average Mt John seeing conditions
magnitude = 5.2 #Change this

if magnitude >= 5.5:
    print(f"magnitude >= 5.5")
    min_exp = 1200  # Minimum exposure time for dim objects
    step = 100  # Larger step size for dim objects
    best_time, snr = find_min_exp_time_for_target_snr(mag=magnitude, seeing=seeing,target_wavelength=target_wavelength,target_snr=target_snr, step=step, min_exp=min_exp, max_exp=1800)
elif 3 <= magnitude < 5.5:
    print(f"magnitude between 3 and 5.5")
    # For brighter objects, start with a smaller minimum exposure time 
    min_exp = 300
    step = 50  # Smaller step size for brighter objects
    best_time, snr = find_min_exp_time_for_target_snr(mag=magnitude, seeing=seeing,target_wavelength=target_wavelength,target_snr=target_snr, step=step, min_exp=min_exp, max_exp=1800)
elif magnitude < 3:
    print(f"Magnitude < 3")
    # For very bright objects, start with a very small minimum exposure time
    min_exp = 10
    step = 10
    best_time, snr = find_min_exp_time_for_target_snr(mag=magnitude, seeing=seeing,target_wavelength=target_wavelength,target_snr=target_snr, step=step, min_exp=min_exp, max_exp=1800)



print(f"\nSelected EXP_TIME = {best_time}s gives S/N = {snr:.2f} at 560nm")

if best_time:
    print(f"\nOptimal exposure time: {best_time} seconds for S/N ≈ {target_snr} at {target_wavelength} nm.")
else:
    print("Could not determine a suitable exposure time.")

end_time = time.time()
time_taken = (end_time - start_time)

print(f"ETC run time: {time_taken:.2f} Seconds")

magnitude between 3 and 5.5
Use OpenMP version of gsetc with 16 threads
##### starting to run ETC ... (it takes a few min.) #####
Compiler flags: -DHGCDTE_SUTR -DMOONLIGHT_
Fiber aperture factor [@800nm, r_eff=0.30"(exp)] = 0.76547869
Fiber aperture factor [@800nm,     point source] = 0.80045737
(1/4) Computing noise vector ...
 //Arm0//
  --> Computing Sky Lines Contribution ...
  --> Computing Sky Continuum Contribution ...
  --> Computing Sky Systematic Error Contribution ...
 --> Done.
(2/4) Computing SNR curve for [OII] emission lines ...
 --> Done.
(3/4) Computing SNR curve for a single line with f=1.00e-17 [erg cm-2 s-1], sigma=70 [km s-1] ...
 --> Done.
(4/4) Computing SNR curve for continuum ...

 --> Done.
OUTFILE_SNL is not found ...
OUTFILE_OII is not found ...
##### finished (elapsed_time: 2.4[sec]) #####
EXP_TIME = 300s --> S/N = 527.25
Use OpenMP version of gsetc with 16 threads
##### starting to run ETC ... (it takes a few min.) #####
Compiler flags: -DHGCDTE_SUTR -DMOO